In [1]:
import multiprocessing as mp
import os
import numpy as np
import pandas as pd
#from matplotlib import pyplot as plt
from keras.callbacks import History, EarlyStopping, Callback
import tensorflow as tf
print(tf.version.VERSION)
import glob as glb



2.9.1


In [2]:
!pwd

/home/jmeow/git/ANNA-MOLLY


In [5]:
os.chdir('telemanom')
os.getcwd()

'/home/jmeow/git/ANNA-MOLLY/telemanom/telemanom'

In [6]:
from channel import Channel
from helpers import Config
import random

In [7]:
cfg = Config("config.yaml")

In [8]:
def get_rando_data(n = 5):
    ret = []
    g = glb.glob("../data/train/*")
    sel = random.choice(g).split("/")[-1].split(".")[0]
    os.chdir('../')
    for _ in range(n):
        ch = Channel(cfg, sel)
        ch.load_data()
        ret.append(ch)
    os.chdir('telemanom')
    return ret



In [9]:
ch = get_rando_data()

In [29]:
from model_utils import TimeGAN

In [10]:
c = ch[1]
nfeatures = c.X_train.shape[-1]

In [31]:
c.shape

AttributeError: 'Channel' object has no attribute 'shape'

In [20]:
from

/home/jmeow/git/ANNA-MOLLY/telemanom/telemanom


In [32]:
c.X_train.shape

(2620, 250, 25)

In [57]:
from keras.layers import LSTM, Input, Concatenate, Reshape, GRU
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, Flatten, RepeatVector
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential, Model
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam


In [95]:
X = Input(shape=(250, 25), name='RealData')
Z = Input(shape=(250, 25), name='RandomData')

In [ ]:
seq_len = 1
n_seq = 25
batch_size = 250
hidden_dim = 10

In [96]:
def make_rnn(n_layers, hidden_units, output_units, name):
    return Sequential([GRU(units=hidden_units,
                           return_sequences=True,
                           name=f'GRU_{i + 1}') for i in range(n_layers)] +
                      [Dense(units=output_units,
                             activation='sigmoid',
                             name='OUT')], name=name)

In [97]:
embedder = make_rnn(n_layers=3,
                    hidden_units=hidden_dim,
                    output_units=hidden_dim,
                    name='Embedder')
recovery = make_rnn(n_layers=3,
                    hidden_units=hidden_dim,
                    output_units=n_seq,
                    name='Recovery')

In [98]:
generator = make_rnn(n_layers=3,
                     hidden_units=hidden_dim,
                     output_units=hidden_dim,
                     name='Generator')
discriminator = make_rnn(n_layers=3,
                         hidden_units=hidden_dim,
                         output_units=1,
                         name='Discriminator')
supervisor = make_rnn(n_layers=2,
                      hidden_units=hidden_dim,
                      output_units=hidden_dim,
                      name='Supervisor')

In [99]:
train_steps = 10000
gamma = 1
mse = MeanSquaredError()
bce = BinaryCrossentropy()

In [100]:
H = embedder(X)
X_tilde = recovery(H)

autoencoder = Model(inputs=X,
                    outputs=X_tilde,
                    name='Autoencoder')

In [101]:
autoencoder.summary()

Model: "Autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RealData (InputLayer)       [(None, 250, 25)]         0         
                                                                 
 Embedder (Sequential)       (None, 250, 10)           2540      
                                                                 
 Recovery (Sequential)       (None, 250, 25)           2255      
                                                                 
Total params: 4,795
Trainable params: 4,795
Non-trainable params: 0
_________________________________________________________________


In [102]:
autoencoder_optimizer = Adam()

In [120]:
@tf.function
def train_autoencoder_init(x):
    with tf.GradientTape() as tape:
        x_tilde = autoencoder(x)
        embedding_loss_t0 = mse(x, x_tilde)
        e_loss_0 = 10 * tf.sqrt(embedding_loss_t0)

    var_list = embedder.trainable_variables + recovery.trainable_variables
    gradients = tape.gradient(e_loss_0, var_list)
    autoencoder_optimizer.apply_gradients(zip(gradients, var_list))
    return tf.sqrt(embedding_loss_t0)

In [118]:
train_steps = c.X_train.shape[0]

In [121]:
from tqdm import tqdm
i = 0
for step in tqdm(range(train_steps)):
    X_ = c.X_train[i]
    X_ = X_.reshape(1, X_.shape[0], X_.shape[1])
    i +=1
    step_e_loss_t0 = train_autoencoder_init(X_)
    if i % 100 == 0:
        print(('Loss Autoencoder Init', step_e_loss_t0, step))
    #with writer.as_default():
    #    tf.summary.scalar('Loss Autoencoder Init', step_e_loss_t0, step=step)

  4%|▍         | 100/2620 [01:41<43:03,  1.03s/it]

('Loss Autoencoder Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.25249106>, 99)


  8%|▊         | 200/2620 [03:19<44:54,  1.11s/it]

('Loss Autoencoder Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.25461343>, 199)


 11%|█▏        | 300/2620 [04:57<43:24,  1.12s/it]

...

 95%|█████████▌| 2500/2620 [41:47<01:58,  1.01it/s]

('Loss Autoencoder Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.24184862>, 2499)


 99%|█████████▉| 2600/2620 [43:33<00:19,  1.00it/s]

('Loss Autoencoder Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.2423632>, 2599)


100%|██████████| 2620/2620 [43:54<00:00,  1.01s/it]


In [122]:
autoencoder.save('autoencoder')

INFO:tensorflow:Assets written to: autoencoder/assets


INFO:tensorflow:Assets written to: autoencoder/assets


In [124]:
supervisor_optimizer = Adam()

In [125]:
@tf.function
def train_supervisor(x):
    with tf.GradientTape() as tape:
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        g_loss_s = mse(h[:, 1:, :], h_hat_supervised[:, :-1, :])

    var_list = supervisor.trainable_variables
    gradients = tape.gradient(g_loss_s, var_list)
    supervisor_optimizer.apply_gradients(zip(gradients, var_list))
    return g_loss_s

In [127]:
i = 0
for step in tqdm(range(train_steps)):
    X_ = c.X_train[i]
    X_ = X_.reshape(1, X_.shape[0], X_.shape[1])
    i +=1
    step_g_loss_s = train_supervisor(X_)
    if i % int(train_steps/10) == 0:
        print(('Loss Generator Supervised Init', step_g_loss_s, step))
    #with writer.as_default():
    #    tf.summary.scalar('Loss Generator Supervised Init', step_g_loss_s, step=step)

 10%|█         | 262/2620 [03:04<24:26,  1.61it/s]

('Loss Generator Supervised Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.003398405>, 261)


 20%|██        | 524/2620 [06:06<26:32,  1.32it/s]

('Loss Generator Supervised Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.0023030809>, 523)


 30%|███       | 786/2620 [09:05<21:39,  1.41it/s]

...

('Loss Generator Supervised Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.0011175827>, 2095)


 90%|█████████ | 2358/2620 [27:22<02:48,  1.56it/s]

('Loss Generator Supervised Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.0008081817>, 2357)


100%|██████████| 2620/2620 [30:34<00:00,  1.43it/s]

('Loss Generator Supervised Init', <tf.Tensor: shape=(), dtype=float32, numpy=0.00044688376>, 2619)


In [128]:
supervisor.save("supervisor")

INFO:tensorflow:Assets written to: supervisor/assets


INFO:tensorflow:Assets written to: supervisor/assets


In [130]:
E_hat = generator(Z)
H_hat = supervisor(E_hat)
Y_fake = discriminator(H_hat)

adversarial_supervised = Model(inputs=Z,
                               outputs=Y_fake,
                               name='AdversarialNetSupervised')
adversarial_supervised.summary()

Model: "AdversarialNetSupervised"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RandomData (InputLayer)     [(None, 250, 25)]         0         
                                                                 
 Generator (Sequential)      (None, 250, 10)           2540      
                                                                 
 Supervisor (Sequential)     (None, 250, 10)           1430      
                                                                 
 Discriminator (Sequential)  (None, 250, 1)            1991      
                                                                 
Total params: 5,961
Trainable params: 5,961
Non-trainable params: 0
_________________________________________________________________


In [132]:
Y_fake_e = discriminator(E_hat)

adversarial_emb = Model(inputs=Z,
                    outputs=Y_fake_e,
                    name='AdversarialNet')
adversarial_emb.summary()

Model: "AdversarialNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RandomData (InputLayer)     [(None, 250, 25)]         0         
                                                                 
 Generator (Sequential)      (None, 250, 10)           2540      
                                                                 
 Discriminator (Sequential)  (None, 250, 1)            1991      
                                                                 
Total params: 4,531
Trainable params: 4,531
Non-trainable params: 0
_________________________________________________________________


In [133]:
X_hat = recovery(H_hat)
synthetic_data = Model(inputs=Z,
                       outputs=X_hat,
                       name='SyntheticData')
synthetic_data.summary()

Model: "SyntheticData"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RandomData (InputLayer)     [(None, 250, 25)]         0         
                                                                 
 Generator (Sequential)      (None, 250, 10)           2540      
                                                                 
 Supervisor (Sequential)     (None, 250, 10)           1430      
                                                                 
 Recovery (Sequential)       (None, 250, 25)           2255      
                                                                 
Total params: 6,225
Trainable params: 6,225
Non-trainable params: 0
_________________________________________________________________


In [134]:
def get_generator_moment_loss(y_true, y_pred):
    y_true_mean, y_true_var = tf.nn.moments(x=y_true, axes=[0])
    y_pred_mean, y_pred_var = tf.nn.moments(x=y_pred, axes=[0])
    g_loss_mean = tf.reduce_mean(tf.abs(y_true_mean - y_pred_mean))
    g_loss_var = tf.reduce_mean(tf.abs(tf.sqrt(y_true_var + 1e-6) - tf.sqrt(y_pred_var + 1e-6)))
    return g_loss_mean + g_loss_var

In [136]:
Y_real = discriminator(H)
discriminator_model = Model(inputs=X,
                            outputs=Y_real,
                            name='DiscriminatorReal')
discriminator_model.summary()

Model: "DiscriminatorReal"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RealData (InputLayer)       [(None, 250, 25)]         0         
                                                                 
 Embedder (Sequential)       (None, 250, 10)           2540      
                                                                 
 Discriminator (Sequential)  (None, 250, 1)            1991      
                                                                 
Total params: 4,531
Trainable params: 4,531
Non-trainable params: 0
_________________________________________________________________


In [137]:
generator_optimizer = Adam()
discriminator_optimizer = Adam()
embedding_optimizer = Adam()

In [140]:
@tf.function
def train_generator(x, z):
    with tf.GradientTape() as tape:
        y_fake = adversarial_supervised(z)
        generator_loss_unsupervised = bce(y_true=tf.ones_like(y_fake),
                                          y_pred=y_fake)

        y_fake_e = adversarial_emb(z)
        generator_loss_unsupervised_e = bce(y_true=tf.ones_like(y_fake_e),
                                            y_pred=y_fake_e)
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

        x_hat = synthetic_data(z)
        generator_moment_loss = get_generator_moment_loss(x, x_hat)

        generator_loss = (generator_loss_unsupervised +
                          generator_loss_unsupervised_e +
                          100 * tf.sqrt(generator_loss_supervised) +
                          100 * generator_moment_loss)

    var_list = generator.trainable_variables + supervisor.trainable_variables
    gradients = tape.gradient(generator_loss, var_list)
    generator_optimizer.apply_gradients(zip(gradients, var_list))
    return generator_loss_unsupervised, generator_loss_supervised, generator_moment_loss

@tf.function
def train_embedder(x):
    with tf.GradientTape() as tape:
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

        x_tilde = autoencoder(x)
        embedding_loss_t0 = mse(x, x_tilde)
        e_loss = 10 * tf.sqrt(embedding_loss_t0) + 0.1 * generator_loss_supervised

    var_list = embedder.trainable_variables + recovery.trainable_variables
    gradients = tape.gradient(e_loss, var_list)
    embedding_optimizer.apply_gradients(zip(gradients, var_list))
    return tf.sqrt(embedding_loss_t0)

@tf.function
def get_discriminator_loss(x, z):
    y_real = discriminator_model(x)
    discriminator_loss_real = bce(y_true=tf.ones_like(y_real),
                                  y_pred=y_real)

    y_fake = adversarial_supervised(z)
    discriminator_loss_fake = bce(y_true=tf.zeros_like(y_fake),
                                  y_pred=y_fake)

    y_fake_e = adversarial_emb(z)
    discriminator_loss_fake_e = bce(y_true=tf.zeros_like(y_fake_e),
                                    y_pred=y_fake_e)
    return (discriminator_loss_real +
            discriminator_loss_fake +
            gamma * discriminator_loss_fake_e)

@tf.function
def train_discriminator(x, z):
    with tf.GradientTape() as tape:
        discriminator_loss = get_discriminator_loss(x, z)

    var_list = discriminator.trainable_variables
    gradients = tape.gradient(discriminator_loss, var_list)
    discriminator_optimizer.apply_gradients(zip(gradients, var_list))
    return discriminator_loss

In [157]:
idata = iter(c.X_train)

In [171]:
next(idata)

array([[-0.98814549,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.98814549,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.98799528,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.98548323,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.98548323,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.98548323,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [173]:
idata = iter(c.X_train)
step_g_loss_u = step_g_loss_s = step_g_loss_v = step_e_loss_t0 = step_d_loss = 0
for step in range(train_steps):
    # Train generator (twice as often as discriminator)
    j = 0
    for kk in range(2):
        X_ = next(idata)
        X_ = X_.reshape(1, X_.shape[0], X_.shape[1])
        Z_ = np.random.rand(250,25)

        # Train generator
        step_g_loss_u, step_g_loss_s, step_g_loss_v = train_generator(X_, Z_)
        # Train embedder
        step_e_loss_t0 = train_embedder(X_)

    X_ = next(idata)
    X_ = X_.reshape(1, X_.shape[0], X_.shape[1])
    Z_ = np.random.rand(250,25)
    step_d_loss = get_discriminator_loss(X_, Z_)
    if step_d_loss > 0.15:
        step_d_loss = train_discriminator(X_, Z_)

    if step % 1000 == 0:
        print(f'{step:6,.0f} | d_loss: {step_d_loss:6.4f} | g_loss_u: {step_g_loss_u:6.4f} | '
              f'g_loss_s: {step_g_loss_s:6.4f} | g_loss_v: {step_g_loss_v:6.4f} | e_loss_t0: {step_e_loss_t0:6.4f}')

    with writer.as_default():
        tf.summary.scalar('G Loss S', step_g_loss_s, step=step)
        tf.summary.scalar('G Loss U', step_g_loss_u, step=step)
        tf.summary.scalar('G Loss V', step_g_loss_v, step=step)
        tf.summary.scalar('E Loss T0', step_e_loss_t0, step=step)
        tf.summary.scalar('D Loss', step_d_loss, step=step)

ValueError: in user code:

    File "/tmp/ipykernel_540553/3486469369.py", line 4, in train_generator  *
        y_fake = adversarial_supervised(z)
    File "/home/jmeow/miniconda3/envs/meow2/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/jmeow/miniconda3/envs/meow2/lib/python3.10/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" is '

    ValueError: Input 0 of layer "AdversarialNetSupervised" is incompatible with the layer: expected shape=(None, 250, 25), found shape=(250, 25)
